In [1]:
#!pip install biopython
#!pip install pandas
import pandas as pd
import numpy as np
import Bio as bio
from Bio.Seq import Seq

In [2]:
def FastaReader(fastafile):
    from Bio import SeqIO
    with open(fastafile) as fasta_file:  # Will close handle cleanly
        identifiers = []
        lengths = []
        seqs = []
        for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
            identifiers.append(seq_record.id)
            seqs.append(seq_record.seq)
        df = pd.DataFrame(identifiers)
        df["Sequence"] = seqs
    return df

In [3]:
genomes = FastaReader('./ncbi_dataset/data/genomic.fna')
genomes.index = genomes[0]
genomes.drop(columns = 0, inplace = True)
#genomes.to_csv('./genomes.csv')
#We may not be using proteins
#proteins = FastaReader('./ncbi_dataset/data/protein.faa')

In [7]:
FastaReader('./test.txt')
#genomes

,0,Sequence
0,this_is_a_test,"(g, a, a, c, a, a, c, c, t, a, c, t, a, g, t, ..."


## 1) Numpy conversion - Generating Dummies

In [6]:
atcg = ["A", "T", "C", "G"]
genomesplit = pd.DataFrame(genomes['Sequence'].apply(lambda x: list(x)))
genomesplit = pd.DataFrame(genomesplit['Sequence'].to_list())
genomesplit[~genomesplit.isin(atcg)] = None
genomesplit.index = genomes.index
dummygenes = genomesplit.fillna(0)
print('non-nucleotides replaced...')
for n in range(0, len(atcg)):
    dummygenes.replace(atcg[n], (n+1), inplace = True)
    print(atcg[n] + ' replaced with ' + str(n))

non-nucleotides replaced...
A replaced with 0
T replaced with 1
C replaced with 2
G replaced with 3


- Replacing Nucleotides with dummy values:

In [7]:
dummygenes.to_csv('./dummygenes.csv')

## 2) Create aligner function

In [8]:
#Genome comparison without Numpy - slow but functional
import difflib
def genomecompare (y):
    tree = genomes['Sequence'].apply(lambda x: difflib.SequenceMatcher(None, str(y), str(x)).ratio())
    return pd.DataFrame(tree).sort_values(0, ascending = False)

In [9]:
#For example, the first 100 nucleotides from this guy:
align = genomecompare(str(genomes['Sequence'][4][0:100]))
align.head(5)
#Gives a series of scores as to what matches best.

,Sequence
0,
U00735.2,0.000321
NC_048217.1,0.000064
NC_048216.1,0.000000
NC_048214.1,0.000000
NC_048213.1,0.000000


In [ ]:
#This function takes a biopython Seq object and splits it into possible open-read-frames
#May or may not come in handy...
def OrfFinder (Seq, table = 11, min_pro_len = 50):
    for strand, nuc in [(+1, Seq), (-1, Seq.reverse_complement())]:
        for frame in range(3):
            length = 3 * ((len(Seq)-frame) // 3) #Multiple of three
            for pro in nuc[frame:frame+length].translate(table).split("*"):
                if len(pro) >= min_pro_len:
                    print("%s...%s - length %i, strand %i, frame %i" % (pro[:30], pro[-3:], len(pro), strand, frame))

In [51]:
#For a more fully-featured alignment (i.e. for query sequences,) align takes a query gene or piece of a genome
#seq should be a string or Biopython seq object.
from Bio import pairwise2 
def align(query, subject):
    if type(query) == str:
        query.replace(" ", "")
        query = Seq(query)
    if type(subject) == str:
        query.replace(" ", "")
        subject = Bio.Seq.Seq(subject)
    alignment = pairwise2.align.globalxx(subject, query)
    return alignment

## 3) Align viruses to WHU-01

In [53]:
from Bio.pairwise2 import format_alignment
from Bio.Alphabet import generic_dna
whu = genomes.loc["MN988668.1"][0]
query = "CA TTTGAC TTAGGC GACGAG CTTGGC ACTGAT CCTTAT GAAGAT TTTCAA GAAAAC TGGAAC ACTAAA CATAGC AGTGGT GTTACC CGTGAA CTCATG CGTGAG CTTAAC GGAGGG GCATAC ACTCGC TATGTC GATAAC AACTTC TGTGGC CCTGAT GGCTAC CCTCTT GAGTGC ATTAAA GACCTT CTAGCA CGTGCT GGTAAA GCTTCA TGCACT TTGTCC GAACAA CTGGAC TTTATT GACACT AAGAGG GGTGTA TACTGC TGCCGT GAACAT GAGCAT GAAATT GCTTGG TACACG GAACGT TCTGAA AAGAGC TATGAA TTGCAG ACACCT TTTGAA ATTAAA TTGGCA AAGAAA TTTGAC ACCTTC AATGGG GAATGT CCAAAT TTTGTA TTTCCC TTAAAT TCCATA ATCAAG ACTATT CAACCA AGGGTT GAAAAG AAAAAG CTTGAT GGCTTT ATGGGT AGAATT CGATCT GTCTAT CCAGTT GCGTCA CCAAAT GAATGC AACCAA ATGTGC CTTTCA ACTCTC ATGAAG TGTGAT CATTGT GGTGAA ACTTCA TGGCAG ACGGGC GATTTT GTTAAA GCCACT TGCGAA TTTTGT GGCACT GAGAAT TTGACT AAAGAA GGTGCC ACTACT TGTGGT TACTTA CCCCAA AATGCT GTTGTT AAAATT TATTGT CCAGCA TGTCAC AATTCA GAAGTA GGACCT GAGCAT AGTCTT GCCGAA TACCAT AATGAA TCTGGC TTGAAA ACCATT CTTCGT AAGGGT GGTCGC ACTATT GCCTTT GGAGGC TGTGTG TTCTCT TATGTT GGTTGC CATAAC AAGTGT GCCTAT TGGGTT CCACGT GCTAGC GCTAAC ATAGGT TGTAAC CATACA GGTGTT GTTGGA GAAGGT TCCGAA GGTCTT AATGAC AACCTT CTTGAA ATACTC CAAAAA GAGAAA GTCAAC ATCAAT ATTGTT GGTGAC TTTAAA CTTAAT GAAGAG ATCGCC ATTATT TTGGCA TCTTTT TCTGCT TCCACA AGTGCT T"
query = Seq(query).complement()
alignment = align(query, whu)
print(format_alignment(*alignment[0]))

...-ATACCTTACAATAGTGT-AACTTCTTCAATTGTCATTACTT-CAGGTGATGGCACAACAAGTCCTATTTCTGAACAT-GACTACCAGATTGGTGGTTATACTGAAAAA-TGGGAATCTGGAGTAAAAGACTGT-GTTGTATTACACAGTTA-CTTCACTTCAGACT-ATTACCAGCTGTACTCAA-CTCAATTGAGTACAG-ACACTGGTGTTGAACA-TGTTACCTTCTTCATCTACAATAAAATTGTT-GATGAGCCTGAAGAACATGTCCAAAT-TCACACAAT-CGACGGTTCATCCGGAGTT-GTTAATCCAGTAATGGAACCAATTTATGATGAACCGAC-GACGACTACTAGCGTGCCTTTGTAA-GCACAAGCTGATGAGTAC-GAACTTATGTACT-CATTCGTTTCGGAAGAGACAGGTACGTTAATAGTTAATAG-CGTACTTCTTTTTCTTGCTTTCGTGGTATTCT-TGCTAGTTACACTAGCC-ATCCTTACTGCGCTTCGATTGTGTG-CGTACTGCTGCAATA-TTGTTAACGTGAGTCTTGTAAAACCTTCTTTTT-ACGTTTACTCTCGT-GTTAAAAATCTGAATTCTTCTAGAGTT-CCTGATCTTCTG-GTCTAAACGAACTAAATATTATATTAGTTTTTCTGTTTGGAA-CTTTAATTTTAGCCATGGCAGAT-TCCAACGGTACTATTAC-CGTTGAAGAGCTTAAAAAGCTCCTT-GAACAATGGAACCTAGTAATAG-GTTTCCTATTCCTTACATGGATTTGTCTT-CTACAATTTGCCTATGCCAA-CAGGAATAGGTTTTTGTATATAATTAAGTTA-ATTTTCCTCTGGCTGTTATGGCCAGTAACTTT-AGCTTGTTTTGTGCTTGCTGCTGTTTACAGAATAAATTGGATCAC-CGGTGGAATTGCTATC-GCAATGGCTTGTCTTGTAGGCTTGATGTGGCTC-AGCTACTTCATTGCTTCTT

## 4) Clustering and clades

## -Moved to clustering.ipynb